## Deliverable 2. Create a Customer Travel Destinations Map.

In [14]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [16]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Ryotsu,JP,38.0833,138.4333,60.40,58,34,10.65,scattered clouds
1,1,Sitka,US,57.0531,-135.3300,46.29,76,90,9.22,light rain
2,2,Asau,RO,46.4333,26.4000,43.75,79,20,4.32,few clouds
3,3,Albany,US,42.6001,-73.9662,52.81,71,98,5.57,overcast clouds
4,4,Port Alfred,ZA,-33.5906,26.8910,66.78,84,100,31.90,overcast clouds


In [18]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 78
What is the maximum temperature you would like for your trip? 90


In [19]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
8,8,Faanui,PF,-16.4833,-151.7500,80.01,77,66,6.26,light rain
10,10,Atar,MR,20.5169,-13.0499,78.15,50,98,4.14,overcast clouds
17,17,Hilo,US,19.7297,-155.0900,82.13,68,1,8.05,clear sky
20,20,Mahajanga,MG,-15.7167,46.3167,78.82,78,40,4.61,scattered clouds
22,22,Butaritari,KI,3.0707,172.7902,79.30,81,100,8.34,light rain
36,36,Olinda,BR,-8.0089,-34.8553,80.33,69,40,10.36,scattered clouds
44,44,Vaini,TO,-21.2000,-175.2000,78.96,83,20,10.36,few clouds
49,49,Kapaa,US,22.0752,-159.3190,84.18,72,1,5.99,clear sky
52,52,Charters Towers,AU,-20.1000,146.2667,84.00,44,3,17.05,clear sky
54,54,Tecoanapa,MX,16.5167,-98.7500,83.43,77,31,7.61,scattered clouds


In [22]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

Unnamed: 0     0
City           0
Country        0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
dtype: int64

In [23]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()

In [27]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
8,Faanui,PF,80.01,light rain,-16.4833,-151.7500,
10,Atar,MR,78.15,overcast clouds,20.5169,-13.0499,
17,Hilo,US,82.13,clear sky,19.7297,-155.0900,
20,Mahajanga,MG,78.82,scattered clouds,-15.7167,46.3167,
22,Butaritari,KI,79.30,light rain,3.0707,172.7902,
36,Olinda,BR,80.33,scattered clouds,-8.0089,-34.8553,
44,Vaini,TO,78.96,few clouds,-21.2000,-175.2000,
49,Kapaa,US,84.18,clear sky,22.0752,-159.3190,
52,Charters Towers,AU,84.00,clear sky,-20.1000,146.2667,
54,Tecoanapa,MX,83.43,scattered clouds,16.5167,-98.7500,


In [37]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
   
   
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    ## Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
   
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:  
       # r=hotels["results"][0]["name"] 
       # print("testing333",r)
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        #hotels["results"][0]["name"]   
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [39]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()

In [40]:
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
8,Faanui,PF,80.01,light rain,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
10,Atar,MR,78.15,overcast clouds,20.5169,-13.0499,Oumou Elghoura
17,Hilo,US,82.13,clear sky,19.7297,-155.0900,Hilo Hawaiian Hotel
20,Mahajanga,MG,78.82,scattered clouds,-15.7167,46.3167,Restaurant Greedy Coco Lodge Majunga
22,Butaritari,KI,79.30,light rain,3.0707,172.7902,Isles Sunset Lodge
36,Olinda,BR,80.33,scattered clouds,-8.0089,-34.8553,Hotel Albergue de Olinda
44,Vaini,TO,78.96,few clouds,-21.2000,-175.2000,Keleti Beach Resort
49,Kapaa,US,84.18,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
52,Charters Towers,AU,84.00,clear sky,-20.1000,146.2667,Cattleman's Rest Motor Inn
54,Tecoanapa,MX,83.43,scattered clouds,16.5167,-98.7500,Hotel Manglares


In [42]:
# 8a. Create the output File (CSV)

# 8b. Export the City_Data into a csv
hotel_df.to_csv("WeatherPy_vacation.csv")

In [45]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Description</dt><dd>{Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [46]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))